In [29]:
import os
import json
import torch

class CustomDatasetYOLO:
    def __init__(self, folder_path, image_size=416, transforms=None):
        self.folder_path = folder_path
        self.transforms = transforms
        self.image_size = image_size
        self.data_pairs = self._load_data_pairs()

    def _load_data_pairs(self):
        image_files = []
        json_files = {}
        data_pairs = []

        # 폴더를 재귀적으로 검색하여 이미지 파일과 JSON 파일 목록을 생성
        for root, _, files in os.walk(self.folder_path):
            for file in files:
                if file.endswith((".jpg", ".jpeg", ".png")):
                    image_files.append(os.path.join(root, file))
                elif file.endswith(".json"):
                    json_files[os.path.splitext(file)[0]] = os.path.join(root, file)

        # 진행률 표시줄 추가
        for image_file_path in image_files:
            # 이미지 파일 이름에서 확장자를 제외한 부분 가져오기
            image_name = os.path.splitext(os.path.basename(image_file_path))[0]

            # 해당 이미지 파일과 매칭되는 JSON 파일 찾기
            if image_name in json_files:
                json_file_path = json_files[image_name]
                data_pairs.append((image_file_path, json_file_path))
            else:
                print(f"JSON file not found for image: {image_file_path}")

        return data_pairs

    def create_yolo_target(self, json_data):
        print(json_data['images'][0]['id'])
        annotations = json_data['annotations']
        categories = {cat['id']: cat['name'] for cat in json_data['categories']}
        
        # 한글에서 영어로 라벨을 매핑하는 딕셔너리
        label_mapping = {'화방': 1, '줄기': 2, '잎': 3, '열매': 4}  # 예시, 실제 매핑에 맞게 수정 필요
        
        # height = float(json_data['images'][0]['height'])
        height = float(json_data['images'][0]['height'])
        # width = float(json_data['images'][0]['width'])
        width = 1960

        targets = []

        for ann in annotations:
            bbox = ann['bbox']  # 바운딩 박스 좌표
            x_min, y_min, w, h = bbox
            print(bbox)
            x_max = x_min + w 
            print("xmax",x_max)
            y_max = y_min + h
            print("ymax" , y_max)
            obj_name = categories[ann['category_id']]  # 객체 이름

            # 한글 카테고리 이름을 영어로 변환하여 매핑된 정수 레이블 얻기
            label = label_mapping.get(obj_name, -1)  # 없는 경우에 대한 처리

            if label is None:
                # 처리할 수 없는 경우 스킵하거나 예외 처리 필요
                continue

            # 바운딩 박스 좌표 변환 및 정규화
            x_center = (x_min + x_max) / 2.0  # 바운딩 박스 중심 x 좌표
            print("x" , x_center)
            y_center = (y_min + y_max) / 2.0  # 바운딩 박스 중심 y 좌표
            print("y", y_center)
            box_width = w  # 바운딩 박스 너비
            box_height = h  # 바운딩 박스 높이

            # YOLO 모델에 맞게 정규화된 좌표 계산
            x_center /= width
            y_center /= height
            box_width /= width
            box_height /= height

            if not (0 <= x_center <= 1 and 0 <= y_center <= 1 and 0 <= box_width <= 1 and 0 <= box_height <= 1):
                print(f"Invalid normalized coordinates for label {label}: [{x_center}, {y_center}, {box_width}, {box_height}]")
                continue

            # 바운딩 박스 정보를 문자열로 변환하여 리스트에 추가
            target_str = f"{label} {x_center} {y_center} {box_width} {box_height}"
            targets.append(target_str)

        return targets

    def create_label_files(self):
        for img_path, json_path in self.data_pairs:
            parent_folder = os.path.dirname(os.path.dirname(img_path))  # 이미지 파일이 있는 폴더의 상위 폴더 경로
            labels_folder = os.path.join(parent_folder, 'labels')
            os.makedirs(labels_folder, exist_ok=True)  # labels 폴더 생성

            txt_file = os.path.join(labels_folder, f"{os.path.splitext(os.path.basename(img_path))[0]}.txt")
            
            with open(json_path, 'r') as json_file:
                json_data = json.load(json_file)

            targets = self.create_yolo_target(json_data)

            # 이미지 경로와 라벨 정보를 텍스트 파일에 기록
            with open(txt_file, 'w') as f:
                for target in targets:
                    f.write(f"{target}\n")

        print("라벨 파일이 생성되었습니다.")




In [14]:
import os
from PIL import Image, ImageFilter
import json
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures


class CustomDatasetYOLO:
    def __init__(self, dataset_folder, image_size=640):
        self.dataset_folder = dataset_folder
        self.image_size = image_size
        self.data_pairs = self._load_data_pairs()

    def _load_data_pairs(self):
        data_pairs = []
        for split in ['train', 'val', 'test']:
            images_folder = os.path.join(self.dataset_folder, split, 'images')
            json_folder = os.path.join(self.dataset_folder, split, 'JSON')

            for root, _, files in os.walk(images_folder):
                for file in files:
                    if file.lower().endswith((".jpg", ".jpeg", ".png")):
                        image_path = os.path.join(root, file)
                        json_path = os.path.join(json_folder, os.path.splitext(file)[0] + '.json')
                        data_pairs.append((image_path, json_path))

        return data_pairs

    def resize_image_and_annotations(self, image_path, annotations, target_size):
        img = Image.open(image_path)
        img_resized = img.resize(target_size, Image.LANCZOS)  # Resize image using LANCZOS filter

        width_ratio = target_size[0] / img.width  # Calculate width ratio for normalization
        height_ratio = target_size[1] / img.height  # Calculate height ratio for normalization
        
        labels = []
        

        for ann in annotations:
            x, y, w, h = ann['bbox']
            category_id = ann['category_id']

            # Adjust bounding box coordinates to resized image
            x_resized = x * width_ratio
            y_resized = y * height_ratio
            w_resized = w * width_ratio
            h_resized = h * height_ratio

            # Calculate YOLO format normalized coordinates
            x_center = (x_resized + w_resized / 2) / target_size[0]
            y_center = (y_resized + h_resized / 2) / target_size[1]
            box_width = w_resized / target_size[0]
            box_height = h_resized / target_size[1]

            # Convert to YOLO format string
            label_str = f"{category_id} {x_center} {y_center} {box_width} {box_height}"
            labels.append(label_str)

        return img_resized, labels

    def create_label_files(self):
        def process_image(image_path, json_path, split):
            parent_folder = os.path.join(self.dataset_folder, split)
            images_folder = os.path.join(parent_folder, 'images')
            labels_folder = os.path.join(parent_folder, 'labels')
            os.makedirs(images_folder, exist_ok=True)
            os.makedirs(labels_folder, exist_ok=True)

            img_name = os.path.splitext(os.path.basename(image_path))[0]
            txt_file = os.path.join(labels_folder, f"{img_name}.txt")
            img_resized_path = os.path.join(images_folder, f"{img_name}.jpg")

            with open(json_path, 'r') as json_file:
                json_data = json.load(json_file)

            img_resized, labels = self.resize_image_and_annotations(image_path, json_data['annotations'], (self.image_size, self.image_size))

            img_resized.save(img_resized_path)

            with open(txt_file, 'w') as f:
                for label in labels:
                    f.write(f"{label}\n")

        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = []
            for image_path, json_path in self.data_pairs:
                split = image_path.split(os.sep)[-3]  # 'train', 'val', 'test'
                futures.append(executor.submit(process_image, image_path, json_path, split))

            for future in concurrent.futures.as_completed(futures):
                try:
                    future.result()
                except Exception as e:
                    print(f"Exception occurred: {e}")

        print("YOLO 라벨 파일 생성이 완료되었습니다.")


In [15]:
if __name__ == "__main__":
    dataset = CustomDatasetYOLO('./datasets', image_size=640)
    dataset.create_label_files()

YOLO 라벨 파일 생성이 완료되었습니다.


In [2]:
# 사용 예시
folder_path = "./datasets"

dataset = CustomDatasetYOLO(folder_path,image_size=640)
dataset.create_label_files()

YOLO 라벨 파일 생성이 완료되었습니다.


In [6]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device


device(type='cpu')

In [7]:
from ultralytics import YOLO

# Load YOLOv10n model from scratch
model = YOLO("yolov10n.yaml")
model.to(device)
# Train the model



YOLOv10(
  (model): YOLOv10DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track

In [8]:


model.train(data="./farmpj.yaml", epochs=30, imgsz=640,  workers=0)
print(model)

New https://pypi.org/project/ultralytics/8.2.42 available 😃 Update with 'pip install -U ultralytics'
engine\trainer: task=detect, mode=train, model=yolov10n.yaml, data=./farmpj.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, val_period=1, cache=False, device=cpu, workers=0, project=None, name=train19, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sh

train: Scanning C:\Users\user\Documents\GitHub\model_dev\datasets\train\labels.cache... 275 images, 0 backgrounds, 0 corrupt: 100%|██████████| 275/275 [00:00<?, ?it/s]
val: Scanning C:\Users\user\Documents\GitHub\model_dev\datasets\val\labels.cache... 80 images, 0 backgrounds, 0 corrupt: 100%|██████████| 80/80 [00:00<?, ?it/s]

Plotting labels to runs\detect\train19\labels.jpg... 



c:\Users\user\anaconda3\envs\py3_11\Lib\site-packages\ultralytics\utils\plotting.py:647: UserWarning: Glyph 54868 (\N{HANGUL SYLLABLE HWA}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)
c:\Users\user\anaconda3\envs\py3_11\Lib\site-packages\ultralytics\utils\plotting.py:647: UserWarning: Glyph 48169 (\N{HANGUL SYLLABLE BANG}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)
c:\Users\user\anaconda3\envs\py3_11\Lib\site-packages\ultralytics\utils\plotting.py:647: UserWarning: Glyph 51460 (\N{HANGUL SYLLABLE JUL}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)
c:\Users\user\anaconda3\envs\py3_11\Lib\site-packages\ultralytics\utils\plotting.py:647: UserWarning: Glyph 44592 (\N{HANGUL SYLLABLE GI}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)
c:\Users\user\anaconda3\envs\py3_11\Lib\site-packages\ultralytics\utils\plotting.py:647: UserWarning: Glyph 51086 (\N{HANGUL SYLLABLE IP}) missing from font(s) DejaVu Sans.
  plt.save

optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 95 weight(decay=0.0), 108 weight(decay=0.0005), 107 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train19
Starting training for 30 epochs...

      Epoch    GPU_mem     box_om     cls_om     dfl_om     box_oo     cls_oo     dfl_oo  Instances       Size


       1/30         0G       3.32      4.463      4.301      3.148      9.176      4.425         48        640:   6%|▌         | 1/18 [00:12<03:40, 12.98s/it]


IndexError: index 4 is out of bounds for dimension 1 with size 4